# Post Here: The Subreddit Suggester

# Notebook 2: Modeling

> Small Model (limited dataset)

### By _Tobias Reaper_

---

## Notebook outline

* [_Imports and Configuration_](#Imports-and-Configuration)
* [Modeling](#Modeling)
  * [Challenges](#Challenges)
  * [Feature Selection](#Feature-Selection)
  * [Vectorization](#Vectorization)
  * [Baseline](#Baseline)
  * [Multinomial Naive Bayes](#Multinomial-Naive-Bayes)
* [Final Thoughts](#Final-Thoughts)

---

## Imports and Configuration

In [1]:
# === General imports === #
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import janitor

In [2]:
# === ML imports === #
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import chi2, SelectKBest

# === NLP Imports === #
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

In [3]:
# === Configure === #
# nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

# Configure pandas display settings
pd.options.display.max_colwidth = 200

# Set random seed
seed = 92

### Load and split data

In [4]:
# === Load cleaned/pruned dataset === #
rspct = pd.read_csv("assets/data/rspct_small.csv", index_col=0)
print(rspct.shape)
rspct.head()

(91500, 2)


,subreddit,text
0,stepparents,Ex Wants Toddler Son (2M) to Meet Her AP/SO - x-post from /r/divorce Quick background: My soon-to-be ex-wife (26F) and I (27M) have been separated for about 5 months now. She has been in a serious...
1,bigseo,"Do we raise our pricing? I took a management role at an agency. We're way, way under $500/mo for SEO pricing - and I'm embarrassed to say that we're hurting for business. Seems to me that it's a s..."
2,chemistry,"Mac vs. PC? Hello, all! I am currently a senior in high school and in the fall I will be going to SUNY Geneseo, majoring in chemistry and minoring in mathematics. Geneseo requires it’s students to..."
3,migraine,"Beer as an aural abortive? Hiya folks,I've been a migraine sufferer pretty much my whole life. For me intense auras, numbness, confusion, the inability to speak or see is BY FAR the worst aspect o..."
4,MouseReview,Recommend office mouse I was hoping you folks could help me out. Here's my situation and requirements:* I don't play games at all* Budget $30.00 or less* Shape as close to old Microsoft Intellimou...


In [5]:
# === Split up dataset into train and test === #
train, test = train_test_split(rspct, test_size=0.2, random_state=seed)
train, val = train_test_split(train, test_size=0.1, random_state=seed)
train.shape, val.shape, test.shape

((65880, 2), (7320, 2), (18300, 2))

In [6]:
# === Split out feature/target === #
X_train = train["text"]
X_val = val["text"]
X_test = test["text"]

y_train = train["subreddit"]
y_val = val["subreddit"]
y_test = test["subreddit"]

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(65880,) (7320,) (18300,)
(65880,) (7320,) (18300,)


---

## Modeling

In [7]:
# === Encode the target using LabelEncoder === #
# This process naively transforms each class of the target into a number
le = LabelEncoder() # Instantiate a new encoder instance
le.fit(y_train)  # Fit it on training label data

# Transform both using the trained instance
y_train = le.transform(y_train)
y_val = le.transform(y_val)
y_test  = le.transform(y_test)

y_train[:8]

array([ 92, 140,  65,  90, 278,  65, 272, 212])

### Vectorization

At first, the "vocabulary" derived from the corpus using the vectorizer was the largest object when serialized. Luckily, there are many options and parameters available to reduce its size.

I decided to remove stopwords before vectorization in hopes that this would reduce the size of the vector vocabulary. To my initial surprise, removing the stop words (using [NLTK](https://www.nltk.org/)'s list) actually increased the size of the serialized vocab from 59mb to 76mb.

After some consideration, I found this to be a reasonable result. I figured that many of the stop words are short ("I", "me", "my", etc.), and their removal caused the average length of words (and therefore bigrams as well) in the vocab to increase. While this may not account for the entirety of the difference, this provides some intuition as to why there is a difference.

Although the vocab without stop words was larger, I ended up using it anyways because it provided an extra ~0.01 in the precision-at-k score of the final model.

In [8]:
# === Look at lengths of stop words === #
lengths = []
three_or_below = []
for word in stop_words:
    lengths.append(len(word))
    if len(word) <= 4:
        three_or_below.append(len(word))
        
print(f"There are {len(stop_words)} stop words in the list.")
print(f"{len(three_or_below)} are 4 chars long or shorter.")
print(f"Average length is: {np.mean(lengths)}.")

There are 179 stop words in the list.
109 are 4 chars long or shorter.
Average length is: 4.229050279329609.


In [10]:
# === Vectorize! === #
# Extract features from the text data using bag-of-words method
tfidf = TfidfVectorizer(
#     max_features=100000,
    min_df=5,
    ngram_range=(1,2),
    stop_words=stop_words,  # Use nltk's stop words
)

# Fit the vectorizer on the feature column to create vocab (doc-term matrix)
vocab = tfidf.fit(X_train)

# Get sparse document-term matrices
X_train_sparse = vocab.transform(X_train)
X_val_sparse = vocab.transform(X_val)
X_test_sparse = vocab.transform(X_test)

X_train_sparse.shape, X_val_sparse.shape, X_test_sparse.shape

((65880, 150524), (7320, 150524), (18300, 150524))

### Feature Selection

As mentioned previously, the size of the corpus means the dimensionality of the featureset after vectorization will be very high. I passed in 100,000 as the maximum number of features to the vectorizer, limiting the initial size of the vocab. However, the features would have to be reduced more before training the model, as it is generally not good practice to have a larger number of features (100,000) than observations (91,500).

To reduce it down from that 100,000, I used a process called select k best, which does exactly what it sounds like: selects a certain number of the best features. The key aspect of this process is how to measure the value of the features; how to find which ones are the "best". The scoring function I used in this case is called [ch2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html) (chi-squared).

This function calculates chi-squared statistics between each feature and the target, measuring the dependence, or correlation, between them. The intuition here is that features which are more correlated with the target are more likely to be useful to the model.

I played around with some different values for the maximum number of features to be selected. Ultimately, I was once again limited by the size of the free Heroku Dyno, and settled on 20,000. This allowed the deployment to go smoothly while retaining enough information for the model to have adequate performance.

In [11]:
# === Feature Selection === #
selector = SelectKBest(chi2, k=20000)

selector.fit(X_train_sparse, y_train)

X_train_select = selector.transform(X_train_sparse)
X_val_select = selector.transform(X_val_sparse)
X_test_select  = selector.transform(X_test_sparse)

X_train_select.shape, X_val_select.shape, X_test_select.shape

((65880, 20000), (7320, 20000), (18300, 20000))

### Model validation

In this case, the model has a target that it is attempting to predict—a supervised problem. Therefore, the performance can be measured on validation and test sets.

To test out the recommendations I copied some posts and put them through the prediction pipeline to see what kinds of subreddits were getting recommended. For the most part, the predictions were decent.

The cases where the recommendations were a little less than ideal happened when I pulled example posts from subreddits that were not in the training data. The model generally did a good job recommending similar subreddits.

#### Baseline

For the baseline model, I decided to go with a basic random forest. This choice was somewhat arbitrary, though I was curious to see how a random forest would do with such a high target cardinality (number of classes/categories).

The baseline precision-at-k metrics for the random forest on the validation set were .54, .63, and .65, for 'k' of 1, 3, and 5, respectively.

In [12]:
# === Evaluate performance using precision-at-k === #
def precision_at_k(y_true, y_pred, k=5):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred = np.argsort(y_pred, axis=1)
    y_pred = y_pred[:, ::-1][:, :k]
    arr = [y in s for y, s in zip(y_true, y_pred)]
    return np.mean(arr)

In [13]:
# === Baseline RandomForest model === #
rfc = RandomForestClassifier(max_depth=32, n_jobs=-1, n_estimators=200)
rfc.fit(X_train_select, y_train)

RandomForestClassifier(max_depth=32, n_estimators=200, n_jobs=-1)

In [14]:
# === Create predictions for validation set === #
y_pred_proba_rfc = rfc.predict_proba(X_val_select)

# === For each prediction, find the index with the highest probability === #
y_pred_rfc = np.argmax(y_pred_proba_rfc, axis=1)
y_pred_rfc[:10]

array([274, 139, 184,  78,  12, 177, 177, 216,  40,  31])

In [15]:
# === Evaluate precision at k for validation === #
print("Validation scores:")
print("  precision@1 =", np.mean(y_val == y_pred_rfc))
print("  precision@3 =", precision_at_k(y_val, y_pred_proba_rfc, 3))
print("  precision@5 =", precision_at_k(y_val, y_pred_proba_rfc, 5))

Validation scores:
  precision@1 = 0.5401639344262295
  precision@3 = 0.6314207650273224
  precision@5 = 0.6545081967213114


#### Multinomial Naive Bayes

Multinomial naive Bayes is a probabilistic learning method for multinomially distributed data, and one of two classic naive Bayes algorithms used for text classification. I decided to iterate with this algorithm because it is meant for text classification tasks.

The precision-at-k metrics for the final Multinomial naive Bayes model on the validation set were .76, .88, and .9188, for 'k' of 1, 3, and 5, respectively. Performance on the test set was nearly identical: .75, .88, and .9159.

In [16]:
# === Naive Bayes model === #
nb = MultinomialNB(alpha=0.1)
nb.fit(X_train_select, y_train)

MultinomialNB(alpha=0.1)

#### Evaluate on validation set

In [17]:
# === Create predictions for validation set === #
y_pred_proba_val = nb.predict_proba(X_val_select)

# For each prediction, find index with highest probability
y_pred_val = np.argmax(y_pred_proba_val, axis=1)
y_pred_val[:10]

array([274, 139, 255,  78,  12,  17, 151, 216,  40, 171])

In [18]:
# === Evaluate precision at k for validation === #
print("Validation scores:")
print("  precision@1 =", np.mean(y_val == y_pred_val))
print("  precision@3 =", precision_at_k(y_val, y_pred_proba_val, 3))
print("  precision@5 =", precision_at_k(y_val, y_pred_proba_val, 5))

Validation scores:
  precision@1 = 0.7673497267759563
  precision@3 = 0.8866120218579235
  precision@5 = 0.9174863387978142


#### Evaluate on test set

In [19]:
# === Create predictions for test set === #
y_pred_proba_test = nb.predict_proba(X_test_select)

# For each prediction, find index with highest probability
y_pred_test = np.argmax(y_pred_proba_test, axis=1)
y_pred_test[:10]

array([ 97, 199, 116, 249,  43, 203,  41, 275,  96,  27])

In [20]:
# === Evaluate precision at k for test === #
print("Test scores:")
print("  precision@1 =", np.mean(y_test == y_pred_test))
print("  precision@3 =", precision_at_k(y_test, y_pred_proba_test, 3))
print("  precision@5 =", precision_at_k(y_test, y_pred_proba_test, 5))

Test scores:
  precision@1 = 0.7575956284153006
  precision@3 = 0.8867213114754099
  precision@5 = 0.9176502732240437


### Recommendations

The API should return a list of recommendations, not a single prediction. To accomplish this, I wrote a function that returns the top 5 most likely subreddits and their respective probabilities.

In [14]:
# === Function to serve predictions === #
# The main functionality of the predict API endpoint

def predict(title: str, submission_text: str, return_count: int = 5):
    """
    Serve subreddit predictions.
    
    Parameters
    ----------
    title : string
        Title of post.
    submission_text : string
        Selftext that needs a home.
    return_count    : integer
        The desired number of recommendations.

    Returns
    -------
    Python dictionary formatted as follows:
        [{'subreddit': 'PLC', 'proba': 0.014454},
         ...
         {'subreddit': 'Rowing', 'proba': 0.005206}]
    """
    # Concatenate title and post text
    fulltext = str(title) + str(submission_text)
    # Vectorize the post -> sparse doc-term matrix
    post_sparse = vocab.transform([fulltext])
    # Feature selection
    post_select = selector.transform(post_sparse)
    # Generate predicted probabilities from trained model
    proba = nb.predict_proba(post_select)
    # Wrangle into correct format
    proba_dict = (pd
                .DataFrame(proba, columns=[le.classes_])  # Classes as column names
                .T  # Transpose so column names become index
                .reset_index()  # Pull out index into a column
                .rename(columns={"level_0": "name", 0: "proba"})  # Rename for aesthetics
                .sort_values(by="proba", ascending=False)  # Sort by probability
                .iloc[:return_count]  # n-top predictions to serve
                .to_dict(orient="records")
               )
    proba_json = {"predictions": proba_dict}
    
    return proba_json

In [15]:
title_science = """Is there an evolutionary benefit to eating spicy food that lead to consumption across numerous cultures throughout history? Or do humans just like the sensation?"""

post_science = """I love spicy food and have done ever since I tried it. By spicy I mean HOT, like chilli peppers (we say spicy in England, I don't mean to state the obvious I'm just not sure if that's a global term and I've assumed too much before). I love a vast array of spicy foods from all around the world. I was just wondering if there was some evolutionary basis as to why spicy food managed to become some widely consumed historically. Though there seem to

It way well be that we just like a tingly mouth, the simple things in life."""

science_recs = predict(title_science, post_science)
science_recs

{'predictions': [{'name': 'GERD', 'proba': 0.009900622287634142},
  {'name': 'Allergies', 'proba': 0.009287774623361566},
  {'name': 'ibs', 'proba': 0.009150308633162811},
  {'name': 'AskAnthropology', 'proba': 0.009028660140513678},
  {'name': 'fatpeoplestories', 'proba': 0.00851982441049019}]}

In [16]:
# === Test post from r/buildmeapc === #

title_pc = """Looking for help with a build"""

post_pc = """I posted my wants for my build about 2 months ago. Ordered them and when I went to build it I was soooooo lost. It took 3 days to put things together because I was afraid I would break something when I finally got the parts together it wouldn’t start, I was so defeated. With virtually replacing everything yesterday it finally booted and I couldn’t be more excited!"""

post_pc_recs = predict(title_pc, post_pc, 10)
post_pc_recs

{'predictions': [{'name': 'lego', 'proba': 0.008418484170536294},
  {'name': 'rccars', 'proba': 0.008112076951648648},
  {'name': 'MechanicalKeyboards', 'proba': 0.0078335440606017},
  {'name': 'fightsticks', 'proba': 0.007633958584830632},
  {'name': 'Luthier', 'proba': 0.00716176615193545},
  {'name': 'modeltrains', 'proba': 0.007088134228361153},
  {'name': 'cade', 'proba': 0.007058109839673285},
  {'name': 'vandwellers', 'proba': 0.006700262151491209},
  {'name': 'cosplay', 'proba': 0.006536648725434882},
  {'name': 'homestead', 'proba': 0.006166832450007183}]}

In [17]:
# === Example post from 'r/learnprogramming' === #

post_title = """What to do about java vs javascript"""

post = """I am a new grad looking for a job and currently in the process with a company for a junior backend engineer role. I was under the impression that the position was Javascript but instead it is actually Java. My general programming and "leet code" skills are pretty good, but my understanding of Java is pretty shallow. How can I use the next three days to best improve my general Java knowledge? Most resources on the web seem to be targeting complete beginners. Maybe a book I can skim through in the next few days?

Edit:

A lot of people are saying "the company is a sinking ship don't even go to the interview". I just want to add that the position was always for a "junior backend engineer". This company uses multiple languages and the recruiter just told me the incorrect language for the specific team I'm interviewing for. I'm sure they're mainly interested in seeing my understanding of good backend principles and software design, it's not a senior lead Java position."""

# === Test out the function === #
post_pred = predict(post_title, post)  # Default is 5 results
post_pred

{'predictions': [{'name': 'cscareerquestions', 'proba': 0.516989539243874},
  {'name': 'devops', 'proba': 0.031462691062989795},
  {'name': 'interviews', 'proba': 0.02846504725703069},
  {'name': 'datascience', 'proba': 0.024227300545057697},
  {'name': 'bioinformatics', 'proba': 0.017516176338177075}]}

In [18]:
# === Test it out with another dummy post === #

title_book = "Looking for books with great plot twists"

# This one comes from r/suggestmeabook
post2 = """I've been dreaming about writing my own stort story for a while but I want to give it an unexpected ending. I've read lots of books, but none of them had the plot twist I want. I want to read books with the best plot twists, so that I can analyze what makes a good plot twist and write my own story based on that points. I don't like romance novels and I mostly enjoy sci-fi or historical books but anything beside romance novels would work for me, it doesn't have to be my type of novel. I'm open to experience after all. I need your help guys. Thanks in advance."""

# === This time with 10 results === #
post2_pred = predict(title_book, post2, 10)
post2_pred

{'predictions': [{'name': 'suggestmeabook', 'proba': 0.4070015062748489},
  {'name': 'writing', 'proba': 0.14985778378113648},
  {'name': 'eroticauthors', 'proba': 0.07159411817054702},
  {'name': 'whatsthatbook', 'proba': 0.06062653422250441},
  {'name': 'ComicBookCollabs', 'proba': 0.027277418056905547},
  {'name': 'Malazan', 'proba': 0.019514923212723943},
  {'name': 'TheDarkTower', 'proba': 0.017162701613834493},
  {'name': 'DestructiveReaders', 'proba': 0.0151031907793204},
  {'name': 'WoT', 'proba': 0.011165890302931272},
  {'name': 'readyplayerone', 'proba': 0.007566597361383115}]}

### Model deployment

As mentioned, the model, vocab, and feature selector were all serialized using Python's pickle module. In the Flask app, the pickled objects are loaded and ready for use, just like that.

I will go over the details of how the Flask app was set up in a separate blog post.

In [19]:
# === Create pickle func to make pickling (a little) easier === #

def picklizer(to_pickle, filename, path):
    """
    Creates a pickle file.
    
    Parameters
    ----------
    to_pickle : Python object
        The trained / fitted instance of the 
        transformer or model to be pickled.
    filename : string
        The desired name of the output file,
        not including the '.pkl' extension.
    path : string or path-like object
        The path to the desired output directory.
    """
    import os
    import pickle

    # Create the path to save location
    picklepath = os.path.join(path, filename)

    # Use context manager to open file
    with open(picklepath, "wb") as p:
        pickle.dump(to_pickle, p)

In [20]:
# === Picklize! === #
filepath = "./assets/pickles"  # Change this accordingly

# Create directory if doesn't exist
os.makedirs(filepath, exist_ok=True)

# Export LabelEncoder as pickle
picklizer(le, "le.pkl", filepath)

# Export selector as pickle
picklizer(selector, "selector.pkl", filepath)

# Export vectorizer as pickle
picklizer(vocab, "vocab.pkl", filepath)

# Export naive bayes model as pickle
picklizer(nb, "nb.pkl", filepath)

---

## Final Thoughts

For me, the most important and valuable aspects of this project were mainly surrounding the challenge of scope management. I constantly had to ask myself, "What is the best version of this I can create given our limitations?"

At first, I thought it would be feasible to predict all of the 1,000+ subreddits in the data, and wasted hours of valuable time attempting to do so. While I had tested various strategies of reducing the complexity of the model, the performance was rather terrible when it was trained on 100 or less examples of each of the complete list of subreddits.

The data scientist who I primarily worked with (we had one data scientist in addition to him and one other machine learning engineer, both of whom did not contribute significantly to the project) kept telling me that I should try reducing the number of classes first, allowing for more examples of each class and fewer classes for the model to predict.

Ultimately, this is the strategy that worked best, and I wasted valuable time by not listening to him the first few times he recommended that strategy. Good teamwork requires the members being humble and listening, something that I have taken to heart since the conclusion of this project.

### Scope Management, Revisited

As time was very short while building this initial recommendation API, there are many things that we wished we could have done but simply did not have the time. Here are a few of the more obvious improvements that could be made.

The first, and most obvious one, is to simply deploy to a more powerful server, such as one hosted on AWS Elastic Beanstalk or EC2. This way, we could use the entire dataset to train an optimal model without worrying (as much) about memory limits.

Second, I could use a Scikit-learn pipeline to validate and tune hyperparameters using cross-validation, instead of a separate validation set. Also, this pipeline could be serialized as a single large object, rather than as separate pieces (encoder, vectorizer, feature selector, and classifier). As a final note for this particular train of thought, [Joblib](https://joblib.readthedocs.io/en/latest/) could potentially provide more efficient serialization than the Pickle module, allowing a more complex pipeline to be deployed on the same server.

Third, a model could've been trained to classify the input post first into a broad category. Then, some sort of model could be used to to classify into a specific subreddit within that broad category. I'm not sure about the feasibility of the second part of this idea, but thought it could be an interesting one to explore.

Lastly, different classes and calibers of models could have been tested for use in the various steps in the pipeline. In this case, I'm referring primarily to using deep learning/neural networks. For example, word vectors could be generated with word embedding models such as Word2Vec. Or the process could be recreated with a library like PyTorch, and a framework like [FastText](https://github.com/facebookresearch/fastText/).

I plan to explore at least some of these in separate blog posts.

As always, thank you for reading! I'll see you in the next one.